In [1]:
import allutils.graph_generator
from allutils.general_PR import flowPR
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pickle
import uuid
import sys
from allutils.utils_experiments import *

import networkx as nx
import copy
import numpy as np
from datetime import datetime, timedelta
import os.path
import matplotlib.pyplot as plt

In [10]:


#mode = sys.argv[1]
#mode = 'SFree'
mode = 'input100'

do_plot = True
do_dump = False

n = 61 #number of nodes in the graph
iters = 1376 #number of temporal edges in the graph

alpha = 0.85
beta = 0.0
gamma = 1.0

if mode != 'random':
    weights = 'real'
else:
    weights = 'random'

In [11]:
def readRealGraph(filepath):
    edgesTS = []
    nodes = set()
    edges = set()
    lookup = {}

    weights = {}
    c = 0
    with open(filepath, 'r') as fd:
        for line in fd.readlines():

            line = line.strip()
            items = line.split(' ')
            tstamp = ' '.join(items[0:2])
            tstamp = tstamp[1:-1]
            tstamp = datetime.strptime(tstamp, '%Y-%m-%d %H:%M:%S')
            t = items[2:4]
            t = list(map(int, t))
            
            t[0] += 1
            t[1] += 1
            
            if t[0] == t[1]:
                continue
            # t.sort(); #undirected

            if tuple(t) in lookup.keys():
                num = lookup[tuple(t)]
            else:
                num = c
                lookup[tuple(t)] = c
                c += 1

            edgesTS.append((tstamp, tuple(t), num))
            nodes.add(t[0])
            nodes.add(t[1])
            edges.add(tuple([t[0], t[1]]))
            # weights[str(t[0])+','+str(t[1])] = int(items[4])
    fd.close()
    return edgesTS, nodes, edges, weights


In [12]:
edgesTS,nodes, edges, weights_input = readRealGraph(os.path.join('.',  "Data", mode+".txt"))

In [14]:
def getGraph(edgesTS):
    G = nx.DiGraph()
    edges = {}

    for item in edgesTS:
        edge = item[1]
        edges[edge] = edges.get(edge, 0.0) + 1.0

    #nrm = float(sum(edges.values()))
    G.add_edges_from([(k[0], k[1], {'weight': v}) for k, v in edges.items()])
    # G.add_edges_from([tuple(edge)])
    return G

In [15]:
G = getGraph(edgesTS)

In [18]:

G = nx.DiGraph(G)

In [20]:
G.remove_edges_from(nx.selfloop_edges(G))


In [22]:

def getSubgraph(G, N):
    Gcc = sorted([G.subgraph(c) for c in nx.connected_components(
        G.to_undirected())], key=len, reverse=True)
    print(len(Gcc))
    nodes = set()
    i = 0

    while len(nodes) < N:
        s = np.random.choice(Gcc[i].nodes())
        i += 1
        nodes.add(s)
        for edge in nx.bfs_edges(G.to_undirected(), s):
            nodes.add(edge[1])
            if len(nodes) == N:
                break
    return nx.subgraph(G, nodes)


In [24]:

G = getSubgraph(G, n)

6


In [3]:

norm = sum([val for (node, val) in G.out_degree(weight='weight')])
sampling_edges = {e[:-1]: e[-1]['weight']/norm for e in G.edges(data=True)}
stream = [list(sampling_edges.keys())[i] for i in np.random.choice(range(len(sampling_edges)), size=iters,)]# p=sampling_edges.values()
    

6
1 22
2 75
3 60
4 18
5 28
6 64


IndexError: list index out of range

In [ ]:

# basic (degree personalization)
personalization = {k: v / norm for k, v in G.out_degree(weight='weight')}
p_prime_nodes = {i: personalization[i]/G.out_degree(i, weight='weight') for i in G.nodes()}
pr_basic = nx.pagerank(G, alpha=alpha, personalization=personalization, weight='weight')
RS4_basic, current_basic = {}, {}
RS4_basic, current_basic, tau_basic, spearman_basic, pearson_basic, error_basic, x = flowPR(p_prime_nodes, pr_basic, stream, RS4_basic, current_basic, iters = iters, beta = beta, gamma = gamma)


In [ ]:
# no personalization
personalization = {k: 1.0 / G.number_of_nodes() for k in G.nodes()}
p_prime_nodes = {i: personalization[i]/G.out_degree(i, weight='weight') for i in G.nodes()}
pr_nopers = nx.pagerank(G, alpha=alpha, personalization=personalization, weight='weight')
RS4_nopers, current_nopers = {}, {}
RS4_nopers, current_nopers, tau_nopers, spearman_nopers, pearson_nopers, error_nopers, x = flowPR(p_prime_nodes, pr_nopers, stream,RS4_nopers, current_nopers, iters = iters, beta = beta, gamma = gamma)


In [ ]:
# random personalization
personalization = {k: np.random.uniform(1e-5, 1.0) for k in G.nodes()}
personalization = {k: v/ sum(personalization.values()) for k,v in personalization.items()}
p_prime_nodes = {i: personalization[i]/G.out_degree(i, weight='weight') for i in G.nodes()}
pr_rand = nx.pagerank(G, alpha=alpha, personalization=personalization, weight='weight')
RS4_rand, current_rand = {}, {}
RS4_rand, current_rand, tau_rand, spearman_rand, pearson_rand, error_rand, x = flowPR(p_prime_nodes, pr_rand, stream, RS4_rand, current_rand, iters = iters, beta = beta, gamma = gamma)


In [ ]:
if do_dump:        
    unique_out = mode + '_personalization_' + str(uuid.uuid4())
    pickle.dump([pr_basic, RS4_basic, pearson_basic, spearman_basic, tau_basic, error_basic, x,
                    pr_nopers, RS4_nopers, pearson_nopers, spearman_nopers, tau_nopers, error_nopers,
                    pr_rand, RS4_rand, pearson_rand, spearman_rand, tau_rand, error_rand], open(unique_out+".p", "wb"))
if do_plot:

    ordered_pr_ref_basic, ordered_pr_out_basic = get_ordered_ranks(pr_basic, RS4_basic)
    ordered_pr_ref_nopers, ordered_pr_out_nopers = get_ordered_ranks(pr_nopers, RS4_nopers)
    ordered_pr_ref_rand, ordered_pr_out_rand = get_ordered_ranks(pr_rand, RS4_rand)

    plt.rcParams.update({'font.size': 20, 'lines.linewidth': 3})
    plt.rcParams['xtick.labelsize'] = 25
    plt.rcParams['ytick.labelsize'] = 25
    
    
    # scatter plot for rankings
    plt.figure('scatter_' + mode)
    plt.scatter(ordered_pr_ref_basic, ordered_pr_out_basic, s=50, c='b')
    plt.plot(range(1, 101), range(1, 101), 'k--')
    plt.xlabel('static PageRank', fontsize=25)
    plt.ylabel('temporal PageRank', fontsize=25)
    plt.ylim((0, 101))
    plt.xlim((0, 101))
    plt.tight_layout()
    plt.savefig(mode + '_scatter.pdf')

    
    
    plt.figure('convergence_basic_' + mode)        
    plt.plot(x, pearson_basic, 'k--')
    plt.plot(x, spearman_basic, 'k-')
    #plt.plot(x, tau_basic, 'k-.')
    plt.plot(x, error_basic, 'k:')
    plt.xlabel('number of temporal edges', fontsize=25)
    plt.ylim((0, 1))
    plt.legend(['Pearson\'s r', 'Spearman\'s rho', 'error'], loc=0)
    plt.tight_layout()
    plt.savefig(mode+'_convergence_basic.pdf')
    
    
    
    plt.figure('convergence_rand' + mode)
    plt.plot(x, pearson_rand, 'k--')
    plt.plot(x, spearman_rand, 'k-')
    #plt.plot(x, tau_basic, 'k-.')
    plt.plot(x, error_rand, 'k:')
    plt.xlabel('number of temporal edges', fontsize=25)
    plt.ylim((0, 1))
    plt.legend(['Pearson\'s r', 'Spearman\'s rho', 'error'], loc=0)
    plt.tight_layout()
    plt.savefig(mode+'_convergence_rand.pdf')
    #plt.show()
